<font size="8">HLA-A:02*01 selection, OOD dataset</font>

Select all the peptide from the HLA-A:02-01 dataset

In [1]:
import pandas as pd
hla_a2 = pd.read_csv("y:/data/hla_a_02_01.csv")
peptide = hla_a2.peptide
peptide.to_csv("y:/data/hla_a_02_01_peptide.csv", index=False)

Use the peptide csv to cluster the data, go to https://services.healthtech.dtu.dk/services/GibbsCluster-2.0/ and submit the file with the following settings; numer of clusters = 1; use trash cluster to remove outliers = true;

the outliers are used as the OOD dataset, save the file at the place you want

With the returned peptide, retrace which ID belongs to it and seperate the data accordingly 

In [2]:
gibbs_report = pd.read_csv("y:/data/hla_a_02_01_peptide_OOD_1_cluster.csv")
gibbs_report = gibbs_report.rename(columns={"## Alignment cores for group 1":"peptide"})
print(gibbs_report.head())

     peptide
0  VLLNAPSEA
1  NLDDVYSYI
2  MTNNPPIPV
3  NLATSIYTI
4  GLDPTGVAV


In [3]:
train_val = hla_a2[hla_a2['peptide'].isin(gibbs_report.peptide.tolist())].reset_index(drop=True)
test = hla_a2[~hla_a2['peptide'].isin(gibbs_report.peptide.tolist())].reset_index(drop=True)
# train_val["binder"] = train_val.measurement_value.apply(lambda x: int(x < 500))
print(train_val.columns)
# print(train_val.head(5))

Index(['ID', 'allele', 'peptide', 'measurement_value',
       'measurement_inequality', 'measurement_type', 'measurement_kind',
       'measurement_source', 'original_allele', 'db2_folder', 'cluster',
       'anchor_0', 'anchor_1', 'cluster_set_10', 'allele_clustering',
       'binder'],
      dtype='object')


Split the train and validation accordingly

In [13]:
from sklearn.model_selection import train_test_split

def split_data(df, ids):
    training_ids, validation_ids = train_test_split(ids, test_size=0.2, stratify=df.binder, random_state=1)
    return training_ids, validation_ids

train_id, val_id = split_data(train_val, train_val.ID.tolist())

In [14]:
OOD_1_cluster = {"train":train_id,
                "val":val_id,
                "test":test.ID.tolist()}

data_list = [{'datatype': key, 'ID': val} for key, values in OOD_1_cluster.items() for val in values]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

In [15]:
# Create a boolean mask
missing_ids = pd.read_csv("Y:\data\missing_ids\BA_pMHCI_human_quantitative_HLA_A2_missing_graph.csv").ID
mask = ~df['ID'].isin(missing_ids)

# Apply the mask to the DataFrame
df_filtered = df[mask]
print(len(df_filtered))
df_filtered.to_csv('Y:/data/OOD_1_cluster_dataset_IDs_datatype.csv', index=False)

8225


In [16]:
merged_df = pd.merge(df_filtered, hla_a2, on='ID', how='inner')
result_df = merged_df.groupby(['datatype'])['binder'].agg(binders='sum', non_binders=lambda x: len(x) - sum(x))
print(len(merged_df))
# Reset index to make 'datatype' a column
result_df = result_df.reset_index()
result_df.to_csv('Y:/data/OOD_1_cluster_dataset_binder_non_binder.csv', index=False)
# Print the result DataFrame
print(result_df)

8225
  datatype  binders  non_binders
0     test       73         1680
1    train     3214         1965
2      val      800          493
